# Planejamento do projeto

## Resultado final
    - Uma classe em linguagem python em que o usuário digita o símbolo da ação (stock ticker) e recebe uma série 
    de gráficos e análises da empresa selecionada.
    - Quais análises serão entregues?
        . Gráfico interativo com o comportamento das ações no tempo;
        . Relatório com os principais indicadores;
        . Avaliação de investimento, favorável ou desvaforável?
        . Previsão de comportamento?

## Ferramental
    - Utilizar o Jupyter Notebook para a escrita e interpretação do código;
    - Utilizar a biblioteca Selenium ou urllib.request para analisar conteudos de páginas;
    - Utilizar a biblioteca Beautiful Soup (ou lXml e Scrapy) que torna possível ler o conteúdo das páginas HTML;
    - Utilizar a biblioteca Pandas para armazenar o conteúdo das páginas em DataFrames;
    - Utilizar a biblioteca Numpy para realização de operações;
    - Utilizar a biblioteca Plotly para desenhar gráficos interativos;
    - Utilizar a biblioetca pyplot da matplotlib para criação de gráficos e dashboards;
    - Utilizar a biblioteca sklearn para a adaptação de um modelo de previsão de preço de ação.
    
## Processo de desenvolvimento
    - Coletar os dados das páginas que contêm informações sobre ações: https://finance.yahoo.com/ 
        - Dados históricos dos preços das ações
        - Indicadores fundamentalistas das ações 
        - Dados históricos dos fundos 
        - Indicadores dos fundos
    - Organizar os dados em DataFrames
    - Tratar os dados, isto é, retirar dados duplicados, tratar dados faltantes, ajustar indices e headers
    - Gerar as análises dos dados

# Utilizando o URLLIB

In [7]:
# Imports para o desenvolvimento do projeto
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib.request as ur
import requests

In [8]:
# Endereço de uma das páginas a qual fornecerá informações para a análise
url = "https://br.financas.yahoo.com/quote/{}/history?period1={}&period2={}&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true"

# Tratar condições de erro

## Como tratar possíveis condições de erro? 
    - Colocando try except 
    - Colocando assert
    

In [9]:
# Inserir o nome empresa
Input_ticker  = str(input())
ticker = Input_ticker +'.SA'

# Selecionado a data do horizonte de busca de 5 anos 
date2 = np.timedelta64(np.datetime64('today') - np.datetime64('1969-12-31'), 's').astype(int)
date1 = date2 - 86400 * 365 * 5

# Formatando o url
url = "https://br.financas.yahoo.com/quote/{}/history?period1={}&period2={}&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true"
url = url.format(ticker, date1, date2)
print(url)

MGLU3
https://br.financas.yahoo.com/quote/MGLU3.SA/history?period1=1485993600&period2=1643673600&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true


In [10]:
# Adicionar cabeçalho para passar informações adicionais. Deve-se identificar o erro retornado pelo navegador (401 por exemplo
# é um exemplo de permissão negada) e adicionar o cabeçalho correspondente, no caso o site necessitava de alguma identificação 
# para concretizar a requisição, essa informação pode ser o tipo de aplicação o sistema operacional, fornecedores, a versão 
# de agente de usuário requisitante e outros. "Mozilla/5.0" é o token geral que diz que o navegador é compatível com Mozilla.
# Por razões históricas, quase todo navegador envia isso hoje. 
# Fonte: https://developer.mozilla.org/pt-BR/docs/Web/HTTP/Headers/User-Agent
     
response = requests.get(url, headers=headers) #baixa o conteúdo de uma página na web
status = response.raise_for_status()

# Retorna um objeto BeautifulSoup que permite manipular o data structure com mais facilidade
soup = BeautifulSoup(response.text, 'html.parser')

# O título da página HTML
print(soup.title.string)

linhas = [] #lista para armazenar os dados retirados do HTML
for element in soup.find_all(['th', 'td', 'strong']):
    linhas.append(element.string) #retira os strings que acompanham as classes selecionadas
    
# Tratando os strings para obter o dado desejado
linhas = [e for e in linhas if e not in ('Operating Expenses','Non-recurring Events')]
new_lista = list(filter(None,linhas)) #retira os Nones

dado = list(zip(*[iter(new_lista)]*7))
df_dado = pd.DataFrame(dado)

NameError: name 'headers' is not defined

# Raspagem da tabela de preços de ações utilizando o Selenium

In [ ]:
# Imports
import time
import requests
import pandas as pd 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import numpy as np

In [12]:
# Ajuste do url
# inserir o nome empresa


# selecionado a data do horizonte de busca de 5 anos 
date2 = np.timedelta64(np.datetime64('today') - np.datetime64('1969-12-31'), 's').astype(int)
date1 = date2 - 86400 * 365 * 5

url = "https://br.financas.yahoo.com/quote/{}/history?period1={}&period2={}&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true"

# formatando o url da página que será acessada
url = url.format(ticker, date1, date2)

# Instância do navegador
opt = webdriver.ChromeOptions()
opt.headless = True #não mostrar a ação em andamento
# ser = Service(r'\chromedriver.exe')
# driver = webdriver.Chrome(service=ser, options=opt)
driver = webdriver.Chrome(options=opt)

driver.get(url)

time.sleep(5)

html = driver.find_element(By.TAG_NAME, 'html')

# posição do inicial do scroll
last_height = driver.execute_script("return document.body.scrollHeight")
print("last_height: {}".format(last_height))

while (True):
    # posiciona o scroll no final da página
    html.send_keys(Keys.END)
    
    # pausa para carregar a página
    time.sleep(1)

    # atualiza a posição do scroll 
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    print("new_height: {}".format(new_height))
    
    # verifica se houve movimento da página 
    if (new_height == last_height):
        # termina o loop
        break
    else:
        # atualiza o último valor 
        last_height = new_height         

element = driver.find_element(By.TAG_NAME, 'table')
html_content = element.get_attribute('outerHTML')

driver.quit()

# Código HTML da tabela
soup = BeautifulSoup(html_content, 'html.parser')

# Tratando o HTML para gerar a tabela de hostórico do preço da ação
# lista para armazenar o dados que não sejam relacionados com o preço da ação (dividendo e desdobramento)
list_not_related = [] #lista com conteúdo não relacionado com o preço das ações
tuplas_eventos = [] #tuplas data, valor, ocorrencia (dividendo e desdobramento)
for element in soup.find_all('td',"Ta(start) Py(10px)"):
    list_not_related.append(element)
    tuplas_eventos.append((element.previous_element, element.find('strong').string, element.find('span').string))

linhas_tab = [] #lista para armazenar os dados das linhas da tebela
# retira os strings que acompanham as classes selecionadas e filtra os casos não queridos
for element in soup.find_all(['td', 'th']):
    if element in list_not_related:
        del(linhas_tab[-1]) #deleta o elemento anterior
    else:
        linhas_tab.append(element.string)
        
new_lista = list(filter(None, linhas_tab)) #retira os Nones se existirem
stock_data = list(zip(*[iter(new_lista)]*7)) #empacota em listas de 7 elementos 

df_data_stocks = pd.DataFrame(stock_data[1:], columns=stock_data[0][0:7]) 
df_not_related = pd.DataFrame(tuplas_eventos, columns=['Data', 'Valor', 'Tipo'])


# Tratando o DataFrame
# converte a última coluna para inteiro substituindo o ponto
df_data_stocks["Volume"] = df_data_stocks['Volume'].apply(lambda x: int(x.replace(".","")) if x != "-" else x)

# laço para substituir os pontos por vírgulas 
for coluna in df_data_stocks.columns[1:6]:
    df_data_stocks[coluna] = df_data_stocks[coluna].apply(lambda x: float(x.replace(",",".")) if x != '-' else x)

# dicionário para auxiliar na correção das datas
dicio = {"jan.": "01", "fev.": "02", "mar.": "03", "abr.": "04", "mai.": "05", "jun.": "06", "jul.": "07", "ago.": "08", "set.": "09", "out.": "10", "nov.": "11", "dez.": "12"}

# ordenando cada data por Ano-mês-dia, substituindo o nome do mês pelo número correspondente e convertendo para datetime64
df_data_stocks["Data_Time"] = pd.to_datetime(df_data_stocks["Data"].apply(lambda x: x.split(' ')[0:6:2][::-1]).\
                                             apply(lambda x: "".join([x[0], dicio[x[1]], x[2]])), 
                                             format='%Y-%m-%d')

# Rearanjando as colunas do dataframe
cols = df_data_stocks.columns.tolist()
cols = cols[-1:] + cols[:-1]
df_data_stocks = df_data_stocks[cols]
df_data_stocks.index = df_data_stocks['Data_Time'] #faz o índice virar a coluna data_time
df_data_stocks.drop(labels='Data_Time', axis=1)

df_data_stocks

last_height: 583
new_height: 8292
new_height: 12081
new_height: 15870
new_height: 19659
new_height: 23448
new_height: 27237
new_height: 31026
new_height: 34816
new_height: 38605
new_height: 42394
new_height: 46183
new_height: 48316
new_height: 48316


,Data_Time,Data,Abrir,Alto,Baixo,Fechamento*,Fechamento ajustado**,Volume
Data_Time,,,,,,,,
2022-01-31,2022-01-31,31 de jan. de 2022,6.80,7.11,6.67,7.00,7.00,149358700
2022-01-28,2022-01-28,28 de jan. de 2022,7.21,7.23,6.62,6.65,6.65,149583500
2022-01-27,2022-01-27,27 de jan. de 2022,6.91,7.28,6.88,7.22,7.22,161788200
2022-01-26,2022-01-26,26 de jan. de 2022,6.95,7.08,6.60,6.75,6.75,135589900
2022-01-25,2022-01-25,25 de jan. de 2022,6.36,6.76,6.20,6.72,6.72,172082000
...,...,...,...,...,...,...,...,...
2017-02-08,2017-02-08,08 de fev. de 2017,0.55,0.55,0.53,0.54,0.51,20505600
2017-02-07,2017-02-07,07 de fev. de 2017,0.55,0.56,0.54,0.55,0.52,14233600
2017-02-06,2017-02-06,06 de fev. de 2017,0.49,0.54,0.49,0.54,0.52,24883200


# Tabela de indicadores de FII

In [13]:
url_FII = 'https://www.fundsexplorer.com.br/ranking'
header ={'user-agent':'Mozilla/5.0'}
lendo_url = requests.get(url_FII, headers = header)

lendo_url2 = lendo_url.text
# lendo_url2
soup = BeautifulSoup(lendo_url.content,'html.parser')
listona = []

for i in soup.find_all('td'):
    listona.append(i.string)
indice = 0
listona_separado_linhas = []
while indice < len(listona):
    lista_provisoria=[]
    for z in range(indice,indice+25):
        lista_provisoria.append(listona[z])
    listona_separado_linhas.append(lista_provisoria)
    indice += 26
# listona_separado_linhas


df_fundos = pd.DataFrame(listona_separado_linhas, columns =['Código do fundo', 'Setor', 'Preço atual','Liquidez','Dividendo','Dividend Yield','DY 3M','DY 6M','DY 12M','DY 3M media','DY 6M media','DY 12M media','DY ANO','Variação Preço','Rentab. Período','Retab. Acum.','Patrimônio Líq.','VPA','P/VPA','DY PATR.','Varia. Patri.','Rentab. Patr.', 'Vacância Física','Vacância Financeira','Quantidade de ativos'])

df_fundos.style

,Código do fundo,Setor,Preço atual,Liquidez,Dividendo,Dividend Yield,DY 3M,DY 6M,DY 12M,DY 3M media,DY 6M media,DY 12M media,DY ANO,Variação Preço,Rentab. Período,Retab. Acum.,Patrimônio Líq.,VPA,P/VPA,DY PATR.,Varia. Patri.,Rentab. Patr.,Vacância Física,Vacância Financeira,Quantidade de ativos
0,APTO11,Híbrido,"R$ 10,10",28.0,"R$ 0,03","0,30%",N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,"R$ 39.476.292,41","R$ 9,84","1,03",N/A,N/A,N/A,N/A,N/A,N/A
1,FIVN11,Shoppings,"R$ 2,40",7307.0,"R$ 0,00","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%",N/A,"0,00%","0,00%","0,00%","R$ 65.805.766,28","R$ 6,99","0,34",N/A,N/A,N/A,N/A,"56,00%",N/A
2,BZLI11,Títulos e Val. Mob.,"R$ 14,70",44041.0,"R$ 0,00","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,68%","0,68%","1,14%","R$ 432.739.853,66","R$ 10,89","1,35",N/A,N/A,N/A,N/A,N/A,N/A
3,XTED11,Lajes Corporativas,"R$ 6,27",164.0,"R$ 0,00","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","1,10%","1,10%","-37,83%","R$ 27.341.371,70","R$ 14,12","0,44",N/A,N/A,N/A,N/A,"0,00%","100,00%"
4,ALMI11,Lajes Corporativas,"R$ 1.002,48",182.0,"R$ 0,00","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","2,27%","2,27%","-28,29%","R$ 250.264.016,46","R$ 2.251,04","0,45",N/A,N/A,N/A,N/A,"68,15%",N/A
5,PABY11,Híbrido,"R$ 9,20",32.0,"R$ 0,00","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","-19,84%","-19,84%","-31,62%","R$ -8.246.025,21","R$ -10,87","-0,85",N/A,N/A,N/A,N/A,N/A,N/A
6,ORPD11,Títulos e Val. Mob.,"R$ 128,79",1.0,"R$ 1,38",N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,"R$ 44.218.407,47","R$ 152,48","0,84",N/A,N/A,N/A,N/A,N/A,N/A
7,RBLG11,Logística,"R$ 88,75",1.0,"R$ 0,80","0,91%","2,64%","4,60%","0,00%","0,88%","0,77%","0,00%","5,99%","0,89%","1,81%","-14,15%","R$ 66.825.500,17","R$ 104,38","0,85",N/A,N/A,N/A,N/A,"0,00%",N/A
8,QAGR11,Outros,"R$ 50,80",9765.0,"R$ 0,39","0,77%","2,31%","4,53%","8,80%","0,77%","0,76%","0,73%","8,31%","3,02%","3,81%","0,70%","R$ 301.164.497,72","R$ 59,75","0,85",N/A,N/A,N/A,N/A,N/A,N/A
9,GGRC11,Logística,"R$ 115,81",6840.0,"R$ 0,82","0,72%","2,18%","4,24%","7,76%","0,73%","0,71%","0,65%","7,76%","7,83%","8,60%","-12,39%","R$ 1.023.742.704,72","R$ 131,95","0,88",N/A,N/A,N/A,N/A,"0,00%","0,00%"


# Tabela de indicadores fundamentalistas Ações (YAHOO)

In [14]:
import pandas as pd
import ssl

ssl._create_default_https_context = ssl._create_unverified_context 

url_link = "http://br.financas.yahoo.com/quote/{}/key-statistics"


r = requests.get(url_link.format(ticker), headers ={'User-Agent':'Mozilla/5.0'})
read_html_pandas_data = pd.read_html(r.text)

read_html_pandas_data

[                                      0       1
 0  Capitalização de Mercado (em um dia)  44.49B
 1                      Valor da Empresa  47.86B
 2                           P/L Passado   61.93
 3                          P/L Estimado   70.42
 4    Índice PEG (expectativa de 5 anos)     NaN
 5                    Preço/Vendas (ttm)    1.25
 6                     Preço/Livro (mrq)    4.09
 7              Valor da Empresa/Receita    1.33
 8               Valor da Empresa/EBITDA   31.12,
                                      0        1
 0        Beta (mensalmente por 5 anos)      082
 1             Variação de 52 Semanas 3  -73,33%
 2  Variação de 52 Semanas de S&P 500 3   17,44%
 3                 Alta de 52 Semanas 3     2645
 4                Baixa de 52 Semanas 3      562
 5             Média Móvel de 50 Dias 3      678
 6            Média Móvel de 200 Dias 3     1527,
                                        0        1
 0               Volume Médio (3 meses) 3  117,31M
 1            

In [15]:
for i in range(0,10):
    read_html_pandas_data[i]

# Indicadores de Ação (STATUSINVEST)
## Principais: 
### Rentabilidade: ROI, ROIC e Margem Líquida
### Endividamento: Liquidez Corrente
### Crescimento: CAGR
### Preço: P/L

In [16]:
import re
# Aquisição do HTML
url_indicadores = 'https://statusinvest.com.br/acoes/{}'
url_indicadores2 = url_indicadores.format(Input_ticker)


headers = {'User-Agent':'Mozilla/5.0'}
url_requests = requests.get(url_indicadores2,headers = headers)
soup = BeautifulSoup(url_requests.text,'html.parser')

# Localização dos dados alvo
lista_recebe_alvos = []
for i in soup.find_all('div','info special w-100 w-md-33 w-lg-20'):
    lista_recebe_alvos.append((i.find('h3').string, i.find('strong').string))
        
for i in soup.find_all('div',{'title':'Valorização no preço do ativo com base nos últimos 12 meses'}):
    lista_recebe_alvos.append((i.find('h3').string, i.find('strong').string))
    
lista_recebe_alvos

for i in soup.find_all('div',["w-50 w-sm-33 w-md-25 w-lg-50 mb-2 mt-2 item",'w-50 w-sm-33 w-md-25 w-lg-16_6 mb-2 mt-2 item']):
    lista_recebe_alvos.append((i.find('h3').string,i.find('strong').string))
lista_recebe_alvos

df_indices_statusinvest = pd.DataFrame(lista_recebe_alvos, columns = ['Indicador','Valor'])
df_indices_statusinvest


,Indicador,Valor
0,Valor atual,"7,00"
1,Valorização (12m),"-71,90%"
2,D.Y,"0,22%"
3,P/L,"65,87"
4,PEG Ratio,"0,59"
5,P/VP,"4,35"
6,EV/EBITDA,"26,44"
7,EV/EBIT,"46,42"
8,P/EBITDA,"26,30"
9,P/EBIT,"46,18"


# Indicadores de FIIs (STATUSINVEST)


In [17]:
import re
# Aquisição do HTML
url_indicadores = 'https://statusinvest.com.br/fundos-imobiliarios/{}'
url_indicadores2 = url_indicadores.format(Input_ticker)


headers = {'User-Agent':'Mozilla/5.0'}
url_requests = requests.get(url_indicadores2,headers = headers)
soup = BeautifulSoup(url_requests.text,'html.parser')

# Localização dos dados alvo
lista_recebe_alvos = []
for i in soup.find_all('div','info special w-100 w-md-33 w-lg-20'):
    lista_recebe_alvos.append((i.find('h3').string, i.find('strong').string))
        
for i in soup.find_all('div',{'title':'Valorização no preço do ativo com base nos últimos 12 meses'}):
    lista_recebe_alvos.append((i.find('h3').string, i.find('strong').string))

df_indices_statusinvest = pd.DataFrame(lista_recebe_alvos, columns = ['Indicador','Valor'])
df_indices_statusinvest



,Indicador,Valor


In [18]:

import plotly.express as px
import ipywidgets as widgets
from ipywidgets import fixed

df_data_stocks.Data_Time.to_list()
# slider_grafico = widgets.IntRangeSlider(value = [df_data_stocks.Data_Time.min(),df_data_stocks.Data_Time.max()],min = df_data_stocks.Data_Time.min(),
# max = df_data_stocks.Data_Time.max(), step = 1, description = 'Escolhe ai mano', disable = False, continuous_update = False, orientation = 'horizontal')
o = df_data_stocks.Data_Time.to_list()[::-1]

options = [e.strftime(' %d/%m/%Y ') for e in o]

Date_Filter = widgets.SelectionRangeSlider(
    options = options,
    index = (0, len(options)-1),
    description = 've ai',
    disabled = False,
    layout={'width': '500px'})


def print_date_range(date_range):
    print(date_range[0])

tuple_filtro = widgets.interact(
    print_date_range,
    date_range= Date_Filter);



interactive(children=(SelectionRangeSlider(description='ve ai', index=(0, 1241), layout=Layout(width='500px'),…

In [19]:
options = [e.strftime(' %d/%m/%Y ') for e in o]

Date_Filter = widgets.SelectionRangeSlider(
    options = options,
    index = (0, len(options)-1),
    description = 've ai',
    disabled = False,
    layout={'width': '500px'})


def update_grafico(df, Filtro1):
    df_funcao = df[df['Data_Time'][0] > np.datetime64(widgets.interact(
    print_date_range,
    date_range= Date_Filter)) & 
                   df['Data_Time'][1] < np.datetime64(widgets.interact(
    print_date_range,
    date_range= Date_Filter))]
    
update_grafico(df_data_stocks, Date_Filter)
  

interactive(children=(SelectionRangeSlider(description='ve ai', index=(0, 1241), layout=Layout(width='500px'),…

ValueError: Could not convert object to NumPy datetime

In [ ]:
np.datetime64(Date_Filter[0].replace('/','-')[::-1]

In [ ]:
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import fixed

slider_grafico = widgets.IntRangeSlider(value = [df_data_stocks.Data_Time.min(),df_data_stocks.Data_Time.max()],min = df_data_stocks.Data_Time.min(),
max = df_data_stocks.Data_Time.max(), step = 1, description = 'Escolhe ai mano', disable = false, continuous_update = false, orientation = 'horizontal')

tratamento_df_grafico = 

n = px.line(df_data_stocks, 'Data_Time',['Fechamento ajustado**','Alto'])
n

In [ ]:
import dash
import dash_core_components as dcc
from dash import html
import plotly.express as px
import pandas as pd

aplicativo = dash.Dash(__name__)

aplicativo.layout = html.Div(children=[
    html.H1(children='Dash de investimentos'),
    html.Div(children='Nesse gráfico é apresentado o preço histórico da ação'),
    dcc.Graph(
    id = 'Gráfico top',
    figure = n
    )
])


if __name__ == '__main__':
    aplicativo.run_server(debug=True)


In [11]:
%tb

NameError: name 'headers' is not defined